<a href="https://colab.research.google.com/github/Psyclophe/Data-Scientist/blob/main/Wal_mart_analysis_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
from pandas.core.indexes.datetimes import date
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels as sm
import sklearn as sl

!pip install pmdarima

import pmdarima as pm
from pmdarima.arima import auto_arima
from pmdarima.model_selection import train_test_split
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as multi

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as mutti

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.compat.scipy import _next_regular

import warnings
warnings.filterwarnings ("ignore")

from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

## sección 1.***Obtención de Datos***
es este apartado se sube la información del dataFrame en github





In [ ]:
#se cargan los datos de entrada proporcionado en el proyecto de las tiendas Wal-mart, asimismo se hace la conexión de los datos a Github mostrado en la siguiente ruta
#de la misma forma se indica a Python muestre los primeros "15" registros

url = 'https://raw.githubusercontent.com/Psyclophe/Data-Scientist/main/Walmart_Store_sales.csv'
df = pd.read_csv (url)
print('Shape of data',df.shape)
df.head(15)

## sección 2. **Análisis Exploratorio**
en esta sección se procede a realizar el análisis de la información proporcionada con Python

In [ ]:
#para identificar el tipo de datos se pide a Python que nos muestre los diferentes tipos datos y, poder detectar la cantidad de datos nulos
df.isnull().sum()

In [ ]:
#con la carga de informacion y con esta funcion se pide a Python indicar el tipo de dato de cada columna, para analizar si es necesario hacer cambios en el tipo de 
#dato en el dataframe y poder tener datos homogeneos se identifico que "Date" es un dato que es objeto y se debe convertir a dato fecha
df.info()

In [ ]:
df.dtypes

In [ ]:
#en esta funcion se pide a Python convertir los datos objeto a string aquí podemos visualizar que el dato que era un objeto "Date" se conviritio al formato fecha 
#en donde se visualiza "datetime64[ns]"
df["Date"]=pd.to_datetime(df["Date"])
df.info()

In [ ]:
#con este script se pide a Pyhon poner en formato a las cifras de la informacion de dataset
pd.options.display.float_format='{:,.2f}'.format

In [ ]:
#con esta funcion se da la instrucción a Python para ver las medidas de tendencia central de las 8 columnas
df.describe()

In [ ]:
#se da la instruccion a Python que se ordene las fechas de menor a mayor en el archivo que se va a analizar asimismo se indica al programa que ordene por fechas, 
#de la mas antigua hasta la mas reciente
data_sort=df.sort_values("Date")
data_sort

In [ ]:
#con este script se pide a Python separar el dato "Date" en year, quarter, month, week & day,
df["Year"]= df['Date'].dt.year
df["Quarter"]= df['Date'].dt.quarter
df["Month"]= df['Date'].dt.month
df["Week"]= df['Date'].dt.week
df["Day"]= df['Date'].dt.day
df

In [ ]:
#con la funcion "groupby" se indica a Python agrupar y sumar, por tienda y por semana, al ejecutar el script Python agrega e indexa la columna "0" a la columna "1" 
#como identificador
group=df.groupby("Store").sum("Weekly_Sales")
group.reset_index(level=0, inplace=True)
group["Weekly_Sales"]=(group["Weekly_Sales"])
group

In [ ]:
sort_WeeklySales=group.sort_values('Weekly_Sales', ascending=False)
sort_WeeklySales

In [ ]:
Top_five=sort_WeeklySales[['Weekly_Sales', 'Store']][0:5:].reset_index(drop=True)
Top_five

In [ ]:
#con este script se pide a Python agrupar por Year
group=df.groupby("Year").sum("Weekly_Sales")
group.reset_index(level=0, inplace=True)
group["Weekly_Sales"]=(group["Weekly_Sales"])
group

In [ ]:
#con este script se pide a Python agrupar por Quarter
group=df.groupby("Quarter").sum("Weekly_Sales")
group.reset_index(level=0, inplace=True)
group["Weekly_Sales"]=(group["Weekly_Sales"])
group

In [ ]:
#con este script se pide a Python ordenar por Store 
df.set_index('Store')

In [ ]:
#con este script se pide a Python graficar las ventas por año y por mes
plt.figure(figsize=(20,8))
sns.lineplot(data=df, x='Month', y='Weekly_Sales', hue='Year', palette='tab10')
sns.set_style('whitegrid')
sns.despine(left=True, offset=20, trim=True)
plt.show()

In [ ]:
#import cufflinks as cf
#from IPython.display import display, HTML

#cf.set_config_file(sharing='public', theme='ggplot', offline=True)
#cf.getThemes()

In [ ]:
#df_pivot = df.pivot(index='Weekly_Sales', columns = 'Month',
#              values = 'Store')

In [ ]:
#df_pivot = df_pivot[['Month','Store]]
#df_pivot.iplot(kind='line'


In [ ]:
#con este script se agrupan las ventas por el segundo y tercer Q de 2012
Q2_sales = df[(df['Date'] >= '2012-04-01') & (df['Date'] <= '2012-06-30')].groupby('Store')['Weekly_Sales'].sum()
Q3_sales = df[(df['Date'] >= '2012-07-01') & (df['Date'] <= '2012-09-30')].groupby('Store')['Weekly_Sales'].sum()

#con este script se grafica la diferencia en ventas por el segundo y tercer Q
plt.figure(figsize=(20,8))
Q2_sales.plot(ax=Q3_sales.plot(kind ='bar'),kind='bar',color='g',alpha=0.2,legend=True)
plt.legend(["Q3' 2012", "Q2' 2012"])

In [ ]:
#en este script se determina el crecimiento entre los Q2 y Q3
Q2_sales= df[(df['Date'] >= '2012-04-01') & (df['Date'] <= '2012-06-30')].groupby('Store')['Weekly_Sales'].sum()
Q3_sales= df[(df['Date'] >= '2012-07-01') & (df['Date'] <= '2012-09-30')].groupby('Store')['Weekly_Sales'].sum()
quarterly_growth_rate = ((Q3_sales - Q2_sales )/Q2_sales)*100
quarterly_growth_rate.sort_values(ascending=False).head()

In [ ]:
plt.figure(figsize=(20,8))
quarterly_growth_rate.sort_values(ascending=False).plot(kind='bar')

In [ ]:
#df=pd('Quarter')
#df_Quarter=df["Date"].dt.quarter
#Period=df[period.third_quarter('Date')]
#df = df.index.quarter
#group=df.groupby(df.index.quarter).head(1).reset_index()
#group

In [ ]:
#se solicita a Python ordenar por tienda y por volumen de ventas, del valor mayor al menor
sort_WeeklySales=group.sort_values('Weekly_Sales', ascending=False)
sort_WeeklySales

In [ ]:
Total_sales= df.groupby('Store')['Weekly_Sales'].sum().sort_values()
Total_sales_array = np.array(Total_sales)
plt.figure(figsize=(20,8))
plt.xticks(rotation=0)
plt.ticklabel_format(useOffset=False, style='plain', axis='y')
plt.title('Sales by store')
plt.xlabel('Store')
plt.ylabel('Total Sales')
Total_sales.plot(kind='bar')

In [ ]:
#con este script le indicamos a Python que solo nos indique el Top_five de las ventas por tienda

#Top_five=sort_WeeklySales[['Weekly_Sales', 'Store']][0:5:].reset_index(drop=True)
#Top_five

In [ ]:
#Sort_Quarter=Quarter[['Year', 'Quarter']][0:5:].reset_index(drop=True)
#Sort_Quarter


In [ ]:
#es este script se solicita a Python calcular la Desviacion Estandar por tienda, ordenanda de mayor a menor
Store_devstd = df.groupby('Store')['Weekly_Sales'].std()
Store_devstd = pd.DataFrame(Store_devstd)
Store_devstd=Store_devstd.sort_values('Weekly_Sales', ascending=False)
Store_devstd

In [ ]:
#con este script Python hace el calculo de la tienda con más desviacion estandar
top_std = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std().sort_values(ascending=False))
top_std.head(1).index[0] , top_std.head(1).Weekly_Sales[top_std.head(1).index[0]] 

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(df[df['Store'] == top_std.head(1).index[0]]['Weekly_Sales'])
plt.title('The Sales Distribution of Store No.'+ str(top_std.head(1).index[0]))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# con este script se determina el -Coeficiente de Desviación Medio- por tienda el
#se determino dividiendo la DesvStd por tienda entre la media de las tiendas, como sigue coeficiente_DM = Desviación Estandar / media aritmetica * 100
coeficiente_DM = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean()*100)
coeficiente_DM.head(5)

In [ ]:
#coef = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean())
#coef = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean()*100)
#coef = coef.rename(columns={'Weekly_Sales':'Coeficiente de DesvSTD'})
#coef_max = coef.sort_values(by='Coeficiente de DesvStd',ascending=False)
#coef_max.head(5)


coef = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean())
coef = coef.rename(columns={'Weekly_Sales':'Coefficient of mean to standard deviation'})
coef_max = coef.sort_values(by='Coefficient of mean to standard deviation',ascending=False)
coef_max.head(7)

In [ ]:
#con este script se pide a Python graficar la distribucion de las Ventas
plt.figure(figsize=(20,8))
sns.distplot(df[df['Store'] == coef_max.head(1).index[0]]['Weekly_Sales'])
plt.title('Sales distribution of Store No.'+str(coef_max.head(1).index[0]))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#con este script se pide a Python identificar los días festivos
df_Holiday=df[df["Holiday_Flag"]==1]
df_Holiday

In [ ]:
#con esta instruccion se definen los Días festivos que encontro Pyton en el dataframe
Super_Bowl =['2010-12-02', '2011-11-02', '2012-10-02']
Labor_Day =  ['2010-10-09', '2011-09-09', '2012-07-09']
Thanksgivig_Day =  ['2010-11-26', '2011-11-25', '2012-11-23']
Xmas = ['2010-12-31', '2011-12-30', '2012-12-28']

In [ ]:
#con este script se pide a Python sumar los dias festivos
add_days = Super_Bowl + Labor_Day + Thanksgivig_Day + Xmas
add_days
df["Holiday_Flag"] = np.where(df["Holiday_Flag"] == 1, df["Holiday_Flag"], df["Holiday_Flag"].isin(add_days))
df['Holiday_Flag'].value_counts()

In [ ]:
#df.loc[df.Date.isin(Super_Bowl)].Date.unique()

In [ ]:
#df.loc[df.Date.isin(Labor_Day)].Date.unique()

In [ ]:
#df.loc[df.Date.isin(Thanksgivig_Day)].Date.unique()

In [ ]:
#df.loc[df.Date.isin(Xmas)].Date.unique()

In [ ]:
#con esta instruccion se pide a Python determinar la media de ventas de los Dias_Festivos
Sales_Super_Bowl= (pd.DataFrame(df.loc[df.Date.isin(Super_Bowl)]))['Weekly_Sales'].mean()
Sales_Labor_Day = (pd.DataFrame(df.loc[df.Date.isin(Labor_Day)]))['Weekly_Sales'].mean()
Sales_Thanksgivig_Day = (pd.DataFrame(df.loc[df.Date.isin(Thanksgivig_Day)]))['Weekly_Sales'].mean()
Sales_Xmas = (pd.DataFrame(df.loc[df.Date.isin(Xmas)]))['Weekly_Sales'].mean()
{Sales_Super_Bowl, Sales_Labor_Day, Sales_Thanksgivig_Day, Sales_Xmas}

In [ ]:
#con esta instruccion se pide a Python determinar la media de ventas de los Dias_No_Festivos
Sales_not_Holiday = df[df['Holiday_Flag'] == 0 ]['Weekly_Sales'].mean()
{Sales_not_Holiday}

In [ ]:
#con esta instruccion se pide a Python determinar el promedio de ventas 
Average_sales = {'Sales_Super_Bowl' : Sales_Super_Bowl,
                 'Sales_Labor_Day': Sales_Labor_Day,
                 'Sales_Thanksgiving_Day':Sales_Thanksgivig_Day,
                 'Sales_Xmas': Sales_Xmas,
                 'Sales_not_Holiday': Sales_not_Holiday}
Average_sales 

In [ ]:
Sales_Quarter=df.groupby('Quarter')['Weekly_Sales'].sum()
Sales_Quarter

In [ ]:
df.groupby('Quarter').sum()

In [ ]:
plt.figure(figsize=(20,8))
plt.bar(df["Month"],df["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Top_five")
plt.show()

In [ ]:
#Yearly Sales
plt.figure(figsize=(20,8))
df.groupby("Year")[["Weekly_Sales"]].sum().plot(kind='bar',legend=False)
plt.xlabel("Years")
plt.ylabel("Weekly Sales")
plt.title("Yearly view of sales")
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(data=df, x='Quarter', y='Weekly_Sales', hue='Year', palette='tab10')
sns.set_style('whitegrid')
sns.despine(left=True, offset=20, trim=True)
plt.show()

In [ ]:
from pandas.core.groupby.groupby import DataFrame

#con este script se pide a Python realizar la suma y acumulacion por Quarter
pd=DataFrame({'Year', 'Quarter'})
print(df.groupby(['Year', 'Quarter']).sum().groupby(level=[0]).cumsum())

In [ ]:
#con este script se pide a Python sumar el total de ventas por Quarter
Sales_Year=df.groupby('Year')['Weekly_Sales'].sum()
Sales_Year

In [ ]:

plt.figure(figsize=(20,8))
sns.lineplot(data=df, x='Year', y='Weekly_Sales', hue='Quarter', palette='tab10')
sns.set_style('whitegrid')
sns.despine(left=True, offset=20, trim=True)
plt.show()

In [ ]:
#df = df.set_index['Date']
#group=df.groupby(df.index.quarter).head(1).reset_index()
#print('Shape of data',df.shape)
#group

In [ ]:
#con este script se muestra en crecimiento tercer trimestre 2012 por tienda

#third_quarter= df[(df['Date'] >= '2012-07-01') & (df['Date'] <= '2012-09-30')].groupby('Store')['Weekly_Sales'].sum().sort_values()
#plt.figure(figsize=(15,7))
#third_quarter.plot(kind='bar',legend=True)
#plt.ticklabel_format( style='plain', axis='y')
#plt.legend(["3-Quarter' 2012"]);


In [ ]:
#con este script se pide a Python graficar las Ventas mensuales por cada año 
plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2010]["Month"],df[df.Year==2010]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2010")
plt.show()

plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2011]["Month"],df[df.Year==2011]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2011")
plt.show()

plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2012]["Month"],df[df.Year==2012]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2012")
plt.show()

In [ ]:
#con esta instrucción se pide a Python graficar las ventas de los tres años de manera mensual
plt.figure(figsize=(20,8))
sns.lineplot(data=df, x='Month', y='Weekly_Sales', hue='Year', palette='tab10')
sns.set_style('whitegrid')
sns.despine(left=True, offset=20, trim=True)
plt.show()

# **Series de tiempo**

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(df.index, df['Weekly_Sales'])

plt.xlabel('Year', fontsize=14 )
plt.ylabel('Quarter', fontsize=14)
plt.title('Series de tiempo', fontsize=14)
plt.show()

In [ ]:
from scipy import stats
x=['Holiday_Flag']
y=['Weekly_Sales']
stats.kruskal(x,y)

In [ ]:
x=df[df['Holiday_Flag'] == 0]['Weekly_Sales']
y=df[df['Holiday_Flag'] == 1]['Weekly_Sales']
stats.kruskal(x,y)

# sección 3. **Quitar Valores de Rango**
en esta sección del codigo se analiza la información para quitar los valores fuera de rango


In [ ]:
#con este script se grafican y validan los valores atipicos
fig, axis = plt.subplots(4,figsize=(16,16))
X = df[['Temperature','Fuel_Price','CPI','Unemployment']]
for i,column in enumerate(X):
    sns.boxplot(df[column],ax=axis[i])

import warnings
warnings.filterwarnings('ignore')

In [ ]:
walmart_data_clean = df[(df['Unemployment']<10) & (df['Unemployment']>4.5) & (df['Temperature']>10)]
walmart_data_clean

In [ ]:
sns.scatterplot(data=df,x='Temperature', y='Weekly_Sales', hue='Year', style='Year')
#sns.scatterplot(data=df,x='Temperature', y='Weekly_Sales', hue='Year')
plt.axvline(x=10,color='b')

In [ ]:
#sns.scatterplot(data=df,x='Holiday_Flag', y='Temperature')


In [ ]:
sns.scatterplot(data=df,x='Unemployment', y='Weekly_Sales', hue='Year', style='Year')
plt.axvline(x=4.5,color='b')

In [ ]:
#con este script pedimos aPython eliminar los valores atipicos en una nueva función
df_clean = df[(df['Unemployment']<10) & (df['Unemployment']>4.5) & (df['Temperature']>10)]


In [ ]:
walmart_data_clean = df[(df['Unemployment']<10) & (df['Unemployment']>4.5) & (df['Temperature']>10)]
walmart_data_clean

In [ ]:
#lim_inf = Q1 - 1.5(whis) IQR (scipy.stats.iqr)
#iqr = Q3 - Q1
#lim_sup = Q3 + 1.5(whis) IQR

In [ ]:
#Checking data for outliers
fig, axis = plt.subplots(4,figsize=(16,16))
X = df_clean[['Temperature','Fuel_Price','CPI','Unemployment']]
for i,column in enumerate(X):
    sns.boxplot(df_clean[column], ax=axis[i])

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Verificamos ya no tener valores atipicos
fig, axis = plt.subplots(4,figsize=(16,16))
X = df_clean[['Temperature','Fuel_Price','CPI','Unemployment']]
for i,column in enumerate(X):
    sns.boxplot(df_clean[column],ax=axis[i])


In [ ]:
#con este grafico de calor podemos ver las correlaciones de las diferentes columnas
plt.figure(figsize = (12,10))
sns.heatmap(df_clean.corr().abs())    

plt.show()

# sección 4. **Modelos y Pronostico**

en esta sección del codigo se procede a realizar el modelado de la información

In [ ]:
# Linear Regression :
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
X = walmart_data_clean[['Store','Fuel_Price','CPI','Unemployment','Day','Month','Year']]
Y = walmart_data_clean['Weekly_Sales']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
print('Linear Regression:')
print()
reg = LinearRegression()
reg.fit(X_train, Y_train)
Y_pred = reg.predict(X_test)
print('Accuracy:',reg.score(X_train, Y_train)*100)
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))
sns.scatterplot(Y_pred, Y_test)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
print('Random Forest Regressor:')
print()
rfr = RandomForestRegressor()        
rfr.fit(X_train,Y_train)
Y_pred = rfr.predict(X_test)
print('Accuracy:',rfr.score(X_test, Y_test)*100)
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, Y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))
sns.scatterplot(Y_pred, Y_test)


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#sns.distplot=sns.distplot('Quarter')

In [ ]:
import prophet as ppt

In [ ]:
#con este script se solicita a Python calcular el analsis de variaza entre 'Holiday_ Flag' vs 'Weekly_Sales', el modelo indica que la variacion
#es pequeña indicando al modelo que los Holiday no afecta las ventas semanales de manera significativa

model = smf.ols ('Holiday_Flag ~ Weekly_Sales', data = df).fit()
aov_table = anova_lm(model, type=2)

#anova_result = sm.stats.anova_lm(model, type=2)
print(aov_table)

In [ ]:
import scipy.stats as stats
from pmdarima import auto_arima

In [ ]:
#df_arima=df
#df_arima=df_arima.set_index('Date').reset.index()

In [ ]:
#plt.figure(figsize=(20,9))
#df_arima['Weekly_Sales'].plot()
#plt.show()

In [ ]:
#df_arimamonth = df_arima.resample('M').mean()

In [ ]:
#plt.figure(figsize=(20,8))
#df_arimamonth['Weekly_Sales'].plot()
#plt.title('Average Sales - Weekly')
#plt.show()

In [ ]:
#train_data = df_arimamonth[:int(0.7*(len(df_arimamonth)))] 
#test_data = df_arimamonth[int(0.7*(len(df_arimamonth))):]

#print('Train:', train_data.shape)
#print('Test:', test_data.shape)

In [ ]:
#train_data['Weekly_Sales'].plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
#test_data['Weekly_Sales'].plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
#plt.show()

In [ ]:
#from pmdarima.arima import ADFTest
#adf_test = ADFTest(alpha = 0.05)
#adf_test.should_diff (df_arimamonth['Weekly_Sales'])

In [ ]:
#model_auto_arima = auto_arima(train_data['Weekly_Sales'], trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
#                  max_p=20, max_q=20, max_P=20, max_Q=20, seasonal=True,maxiter=200,
#                  information_criterion='aic',stepwise=False, suppress_warnings=True, D=0, max_D=0,
#                  error_action='ignore',approximation = False)
#model_auto_arima.fit(train_data['Weekly_Sales'])

In [ ]:
#model_auto_arima.summary()

In [ ]:
#y_pred = model_auto_arima.predict(n_periods=len(test_data['Weekly_Sales']))
#y_pred = pd.DataFrame(y_pred,index = test_data['Weekly_Sales'].index,columns=['Prediction'])
#plt.figure(figsize=(20,6))
#plt.title('Prediction of Weekly Sales Using Auto-ARIMA', fontsize=20)
#plt.plot(train_data['Weekly_Sales'], label='Train')
#plt.plot(test_data['Weekly_Sales'], label='Test')
#plt.plot(y_pred, label='Prediction of ARIMA')
#plt.legend(loc='best')
#plt.xlabel('Date', fontsize=14)
#plt.ylabel('Weekly Sales', fontsize=14)
#plt.show()

In [ ]:
#con este escript se elige el tipo de modelo

#import statsmodels as sm
#from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.stattools import adfuller
#def test_stationarity(timeseries):

#estadisticas en la determinación de la media móvil y la desviación estandar
#  rolmean = pd.rolling_mean(timeseries, window=12)
#  rolstd = pd.rolling_std(timeseries, window=12)

#graficar las estadisticas
#  orig = plt.plot(timeseries, color="blue", label="Original")
#  mean = plt.plot(rolmean, color="red", label="Media Movil")
#  std = plt.plot(rolstd, color="black", label="Desviacon Estandar")
#  plt.legend(loc="best")
#  plt.title("Media Movil y Desviacion Estandar")
#  plt.show(block=False)

#calculo de la media movil
#moving_avg = pd.rolling_mean(datos["Weekly_Sales"], 12)
#datos["Weekly_Sales"]

#grafica de la media movil
#moving_avg.plot(label="Media Movil", color="red")
#plt.legend(loc="best")

#from statsmodels.tsa.stattools import adfuller
#def test_stationarity(timeseries):
    #determine rolling statistics
#    rolmean = pd.Series(timeseries).rolling(window=24).mean()#24 hours on each day
#    rolstd = pd.Series(timeseries).rolling(window=24).std()
#    #plot rolling statistics
#    orig = plt.plot(timeseries,color = 'blue',label='original')
#    mean = plt.plot(rolmean,color = 'red',label = 'rolling mean')
#    std = plt.plot(rolstd,color = 'black',label = 'rolling std')
#    plt.legend(loc = 'best')
#    plt.title('rolling mean and standard deviation')
#    plt.show(block = False)

   #perform dickey fuller test
#    print('result of dickey fuller test:')
#    dftest = adfuller(timeseries,autolag = 'AIC')
#    dfoutput = pd.Series(dftest[0:4],index = ['Test statistics', 'p-value', '#lags used', 'number of observation used'])
#    for key,value in dftest[4].items():
#        dfoutput['critical value (%s)'%key] = value
#    print(dfoutput)

#from matplotlib.pylab import rcParams
#rcParams['figure.figsize'] = 20,10
#test_stationarity(train_original['Count'])


In [ ]:
#df = px.data.gapminder()
#Weekly_Sales = df.query("Weekly_Sales")

#for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
#    fig = px.scatter(Weekly_Sales,
#                     x="gdpPercap", y="lifeExp", size="pop", color="continent",
#                     log_x=True, size_max=60,
#                     template=template, title="Gapminder 2007: '%s' theme" % template)
#    fig.show()

#df = px.data.gapminder()
#df_2007 = df.query("year==2007")

#for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
#    fig = px.scatter(df_2007,
#                     x="gdpPercap", y="lifeExp", size="pop", color="continent",
#                     log_x=True, size_max=60,
#                     template=template, title="Gapminder 2007: '%s' theme" % template)
#    fig.show()

In [ ]:
#f.["Temperature"].plot(figsize=(20,8))

#df.plot()

In [ ]:
#plt.plot(test)

#train = df_clean['Weekly_Sales']
#test = df_clean['Weekly_Sales']
#plt.plot(train)
#plt.plot(test)

In [ ]:
#plt.figure(figsize=(50,6))
#df_clean['Weekly_Sales'].plot()

In [ ]:
#model_auto_arima = auto_arima(train_data_diff, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
#                  max_p=20, max_q=20, max_P=20, max_Q=20, seasonal=True,maxiter=200,
#                  information_criterion='aic',stepwise=False, suppress_warnings=True, D=1, max_D=10,
#                  error_action='ignore',approximation = False)
#model_auto_arima.fit(train_data_diff)

In [ ]:
#X = df_clean[['Store','Fuel_Price','CPI','Unemployment','Day','Month','Year']]
#y = df_clean['Weekly_Sales']

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

In [ ]:
#from sklearn.linear_model import LinearRegression

#model = LinearRegression()
#model.fit(X_train,y_train)